<a href="https://colab.research.google.com/github/youkyung2/dataAnalysis/blob/master/%EC%9E%91%EC%97%85%ED%98%95_%EC%A0%9C2%EC%9C%A0%ED%98%95_roc_auc_0_72.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 출력을 원하실 경우 print() 활용
# 예) print(df.head())

# 답안 제출 예시
# 수험번호.csv 생성
# DataFrame.to_csv("0000.csv", index=False)

import pandas as pd
'''개꿀팁...컬럼, 인덱스 모두 표시!'''
# pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')

'''1. csv파일 가져오기'''
y_train = pd.read_csv('data/y_train.csv')
X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')


'''15.
훈련&검증용으로 분리할 때 X_test이랑 이름이 겹친다...
그래서 x_test로 하자
'''
x_test = X_test.copy()


# print(y_train)
# print(X_train)
# print(X_test)

'''2. y_train, X_train 병합'''
df = pd.merge(y_train, X_train, on='cust_id')
# print(df)
df.drop(['cust_id'], axis=1, inplace=True)
# print(df)
# print(df.info())

''' 3.
환불금액에 결측값이 많다. 확인 필요
환불금액이 없으면 NaN으로 나온 것 같으니 0으로 대체하자.
'''
# print(df.환불금액.value_counts())
# df.환불금액.replace(np.nan, 0, inplace=True)
df.fillna(0, inplace=True)
# print(df)

'''4.
남녀 성비를 보자
여자 2184, 남자 1316
'''
# print(df.groupby(['gender']).count()['총구매액'])
# print(2184/1316)

'''5.
오브젝트 중 주구매상품을 성별에 따라 탐색해보자
성비를 생각해서 1.66보다 높으면 0, 낮으면 1로 분류 - 별로임
여자는 w, 남자는 m
사분위로 구간을 나눠서 분류하자 - 이게 낫다
여성 분들이 많이 사는 품목일수록 3,
남성 분들이 많이 사는 품목일수록 0이 되도록 함
'''
# print(df.groupby(['gender', '주구매상품']).count())
# print(df.주구매상품.value_counts())
# print(help(df.groupby))
w = df[df.gender==0][['gender', '주구매상품']] # 2차원으로 컬럼명을 묶어서 줘야한다. 
m = df[df.gender==1][['gender', '주구매상품']]
# print(w)
# print(w.groupby(['주구매상품']).count())
w_주구매상품 = w.groupby(['주구매상품']).count().reset_index()
m_주구매상품 = m.groupby(['주구매상품']).count().reset_index()
# print(w_주구매상품)

wm_주구매상품 = pd.merge(w_주구매상품,m_주구매상품,on='주구매상품')
# # print(help(pd.merge))
wm_주구매상품 = wm_주구매상품.rename(columns={'gender_x':'w','gender_y':'m'}).\
							sort_values(by=['w'], ascending=False).reset_index().drop(['index'], axis=1)
# print(help(df.rename))
# print(help(df.sort_values))
# print(wm_주구매상품)
# print(wm_주구매상품['w']/wm_주구매상품['m'])
wm_주구매상품['성비'] = wm_주구매상품['w']/wm_주구매상품['m']
# print(wm_주구매상품.describe())
def 성비_양자화(value):
	if value < 0.926:
		value=0
	elif value < 1.165:
		value=1
	elif value < 1.893:
		value=2
	else:
		value=3
	return value
wm_주구매상품['범주화'] = wm_주구매상품.성비.apply(lambda x:성비_양자화(x))
# wm_주구매상품['범주화'] = pd.qcut(wm_주구매상품['성비'], q=4).cat.codes

# print(wm_주구매상품)

상품0 = wm_주구매상품[wm_주구매상품['범주화']==0]['주구매상품'].tolist()
상품1 = wm_주구매상품[wm_주구매상품['범주화']==1]['주구매상품'].tolist()
상품2 = wm_주구매상품[wm_주구매상품['범주화']==2]['주구매상품'].tolist()
상품3 = wm_주구매상품[wm_주구매상품['범주화']==3]['주구매상품'].tolist()

def 주구매상품_범주화(value):
	if value in 상품0:
		value = 0
	elif value in 상품1:
		value = 1
	elif value in 상품2:
		value = 2
	else:
		value = 3
	return value
df['주구매상품'] = df.주구매상품.apply(lambda x: 주구매상품_범주화(x))
# print(df.주구매상품)

# print(df.corr())
# print(df.info())

'''6.
환불금액을 생각해보면 여자분들이 더 환불을 많이 할 가능성이 있다.
단순히 환불 여부만으로 따지는 것이 맞을 것같다.
'''
df['환불여부'] = df['환불금액']
#df[df['환불여부']!=0]['환불여부']=1 #이건 안된다.
#print(df)

idx = df[df['환불여부']!=0].index
df['환불여부'][idx]=1

# print(df)
# print(df.corr())

'''7.
주말방문비율로 주말에 많이 오면 남자, 적게 오면 여자라고 생각해보자
비율로 따지니 남녀가 갈린다.
그럼 내점일수*(1-주말방문비율)='평일방문일수' 컬럼을 만들어보자 - 좋다
(df.내점일수*(1-df.주말방문비율))-(df.내점일수*df.주말방문비율) - 더 좋다
'''
# print(df.groupby(['gender','주말방문비율']).count())
w = df[df.gender==0]['주말방문비율']
m = df[df.gender==1]['주말방문비율']
# print(w.describe())
# print(m.describe())
df['평일방문가중치'] = (df.내점일수*(1-df.주말방문비율))-(df.내점일수*df.주말방문비율)
# df['주말방문일수'] = (df.내점일수*df.주말방문비율)
# print(df)
# print(df.corr())

'''8.
주구매지점 레이블 인코딩해서 확인해보자
없는 게 낫다
'''
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['주구매지점'] = le.fit_transform(df[['주구매지점']])
# print(df)
# print(df.corr())
df.drop(['주구매지점'], axis=1, inplace=True)


'''9.
민맥스로 스케일링 사용하자
gender는 빼고하자. 스케일링하고 다시 넣자
'''
df.drop(['gender'], axis=1, inplace=True)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(data=scaled, columns=df.columns)
df_scaled = pd.concat([y_train, df_scaled], axis=1)
# print(help(pd.concat))
df_scaled.drop(['cust_id'], axis=1, inplace=True)
# print(df_scaled)

'''10.
describe로 확인하면서 로그 변환 해보자
'''
# print(df_scaled.describe())
# print(df_scaled.describe().T[['mean','50%']])

# print((df_scaled.describe().T['mean']/df_scaled.describe().T['50%'])>1.3)
df_describe = (df_scaled.describe().T['mean']/df_scaled.describe().T['50%'])>1.3

# print(df_describe.index)
# print(df_describe[df_describe.values==True].index)
로그변환목록 = df_describe[df_describe.values==True].index.tolist()
로그변환목록.remove('gender') #완전 개중요!! 꼭 기억하자!

# print(로그변환목록)
for 목록 in 로그변환목록:
	df_scaled[목록] = np.log1p(df_scaled[목록])
# print(df_scaled)
# print(df_scaled.info())
# print(df_scaled.corr())
'''11.
범주화하자
describe로 사분위수 봐서 범주화하자
코드로 사분위수 찾아서 코딩 짜면 시간 오바된다...
그냥 숫자 일일이 대입하자ㅠㅠ
범주화 하기 전후 corr을 비교하여 뺄 건 빼자
gender와 상관계수 낮은 '내점당구매건수','구매주기'은 빼자
'''
# print(df_scaled.describe().T.loc[:,'min':'75%'])
def 총구매액_범주화(value):
	if value < 0.02378:
		value = 0
	elif value < 0.033383:
		value = 1
	elif value < 0.064758:
		value = 2
	else:
		value = 3
	return value
df_scaled['총구매액_범주'] = df_scaled.총구매액.apply(lambda x: 총구매액_범주화(x))


def 최대구매액_범주화(value):
	if value < 0.008234:
		value = 0
	elif value < 0.017917:
		value = 1
	elif value < 0.035922:
		value = 2
	else:
		value = 3
	return value
df_scaled['최대구매액_범주'] = df_scaled.최대구매액.apply(lambda x: 최대구매액_범주화(x))


def 내점일수_범주화(value):
	if value < 0.003515:
		value = 0
	elif value < 0.024349:
		value = 1
	elif value < 0.081126:
		value = 2
	else:
		value = 3
	return value
df_scaled['내점일수_범주'] = df_scaled.내점일수.apply(lambda x: 내점일수_범주화(x))


# def 내점당구매건수_범주화(value):
# 	if value < 0.031131:
# 		value = 0
# 	elif value < 0.061322:
# 		value = 1
# 	elif value < 0.106743:
# 		value = 2
# 	else:
# 		value = 3
# 	return value
# df_scaled['내점당구매건수_범주'] = df_scaled.내점당구매건수.apply(lambda x: 내점당구매건수_범주화(x))


# def 주말방문비율_범주화(value):
# 	if value < 0.027291:
# 		value = 0
# 	elif value < 0.256410:
# 		value = 1
# 	elif value < 0.448980:
# 		value = 2
# 	else:
# 		value = 3
# 	return value
# df_scaled['주말방문비율'] = df_scaled.주말방문비율.apply(lambda x: 주말방문비율_범주화(x))


# def 구매주기_범주화(value):
# 	if value < 0.023811:
# 		value = 0
# 	elif value < 0.075398:
# 		value = 1
# 	elif value < 0.155870:
# 		value = 2
# 	else:
# 		value = 3
# 	return value
# df_scaled['구매주기'] = df_scaled.구매주기.apply(lambda x: 구매주기_범주화(x))


def 평일방문가중치_범주화(value):
	if value < 0.272608:
		value = 0
	elif value < 0.280674:
		value = 1
	elif value < 0.323968:
		value = 2
	else:
		value = 3
	return value
df_scaled['평일방문가중치_범주'] = df_scaled.평일방문가중치.apply(lambda x: 평일방문가중치_범주화(x))

# print(df_scaled)
# print(df_scaled.corr())
# print(df_scaled.corr()>0.8)
df_scaled.drop(['내점당구매건수','구매주기'], axis=1, inplace=True)
# print(help(df.drop))
print(df_scaled.corr())

'''12.
df_scaled을 훈련과 검증용으로 분리
!!!완전중요!!!!! 분류에서 y 타겟은 무조건 정수여야한다.
'''
from sklearn.model_selection import train_test_split
# print(help(train_test_split))
X = df_scaled.drop(['gender'], axis=1)
y = df_scaled['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

'''13.
교차검증과 xgb 가져오자
'''
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
# print(dir(sklearn.model_selection))
xgb = XGBClassifier(n_jobs=-1, random_state=20)
# print(help(XGBClassifier))
params = {'n_estimators':[400], 'max_depth':[2], 'learning_rate':[0.01]}
grid = GridSearchCV(xgb, param_grid=params, n_jobs=-1, cv=3)
# print(help(GridSearchCV))
grid.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid.best_params_)
best_grid = grid.best_estimator_
pred = best_grid.predict(X_test)
pred_proba = best_grid.predict_proba(X_test)[:,1]
# print(pred_proba)


'''14.
평가 함수
'''
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
# print(dir(sklearn.metrics))
def get_clf_eval(y_test, pred, pred_proba):
	accuracy = accuracy_score(y_test, pred)
	confusion = confusion_matrix(y_test, pred)
	roc_auc = roc_auc_score(y_test, pred_proba)
	print('혼동행렬: \n', confusion)
	print(f'정확도: {accuracy}, roc_auc: {roc_auc}')

get_clf_eval(y_test, pred, pred_proba)

'''15.
테스트 데이터에 적용
'''
# print(x_test)
cust_id = x_test.copy()
x_test.drop(['cust_id'], axis=1, inplace=True)

x_test.fillna(0, inplace=True)

def 주구매상품_범주화(value):
	if value in 상품0:
		value = 0
	elif value in 상품1:
		value = 1
	elif value in 상품2:
		value = 2
	else:
		value = 3
	return value
x_test['주구매상품'] = x_test.주구매상품.apply(lambda x: 주구매상품_범주화(x))

x_test['환불여부'] = x_test['환불금액']
idx = x_test[x_test['환불여부']!=0].index
x_test['환불여부'][idx]=1
#df['환불여부'] = df['환불금액'].apply(lambda x : 1 if x != 0 else 0)

x_test['평일방문가중치'] = (x_test.내점일수*(1-x_test.주말방문비율))-(x_test.내점일수*x_test.주말방문비율)

x_test.drop(['주구매지점'], axis=1, inplace=True)

scaled = scaler.transform(x_test)
test_scaled = pd.DataFrame(data=scaled, columns=x_test.columns)

for 목록 in 로그변환목록:
	test_scaled[목록] = np.log1p(test_scaled[목록])

def 총구매액_범주화(value):
	if value < 0.02378:
		value = 0
	elif value < 0.033383:
		value = 1
	elif value < 0.064758:
		value = 2
	else:
		value = 3
	return value
test_scaled['총구매액_범주'] = test_scaled.총구매액.apply(lambda x: 총구매액_범주화(x))


def 최대구매액_범주화(value):
	if value < 0.008234:
		value = 0
	elif value < 0.017917:
		value = 1
	elif value < 0.035922:
		value = 2
	else:
		value = 3
	return value
test_scaled['최대구매액_범주'] = test_scaled.최대구매액.apply(lambda x: 최대구매액_범주화(x))


def 내점일수_범주화(value):
	if value < 0.003515:
		value = 0
	elif value < 0.024349:
		value = 1
	elif value < 0.081126:
		value = 2
	else:
		value = 3
	return value
test_scaled['내점일수_범주'] = test_scaled.내점일수.apply(lambda x: 내점일수_범주화(x))

def 평일방문가중치_범주화(value):
	if value < 0.272608:
		value = 0
	elif value < 0.280674:
		value = 1
	elif value < 0.323968:
		value = 2
	else:
		value = 3
	return value
test_scaled['평일방문가중치_범주'] = test_scaled.평일방문가중치.apply(lambda x: 평일방문가중치_범주화(x))

test_scaled.drop(['내점당구매건수','구매주기'], axis=1, inplace=True)

# print(test_scaled)
# print(test_scaled.info())

test_pred_proba = best_grid.predict_proba(test_scaled)[:,1]
# print('-'*30)
# print(test_pred_proba)
# print('-'*30)
# print(cust_id)

cust_id['gender'] = test_pred_proba
# print(cust_id[['cust_id', 'gender']])

제출용 = cust_id[['cust_id', 'gender']].rename(columns={'cust_id':'custid'})
# print(help(df.rename))
# print(제출용)
제출용.to_csv('data/수험번호.csv')

제출용확인 = pd.read_csv('data/수험번호.csv')
print(제출용확인)